In [ ]:
from mtcnn import MTCNN
from ultralytics import YOLO
import cv2
import json
import os
from config import PHOTO_FOLDER_PATH, MODEL_PATH, PROJECT_PATH

detector = MTCNN()
model = YOLO(MODEL_PATH)

# Create a new folder for saving the mini photo
mini_photo_folder = os.path.join(PHOTO_FOLDER_PATH, "mini_photo")
os.makedirs(mini_photo_folder, exist_ok=True)

# Count the number of images in the folder
num_images = 0
image_names = []  # List to store the image names
no_faces_detected = []  # List to store the names of images with no faces detected

for filename in os.listdir(PHOTO_FOLDER_PATH):
    if filename.endswith(".jpg") or filename.endswith(".jpeg") or filename.endswith(".png"):
        num_images += 1
        image_names.append(filename)

# Iterate over each image in the folder
for i, filename in enumerate(os.listdir(PHOTO_FOLDER_PATH)):
    if filename.endswith(".jpg") or filename.endswith(".jpeg") or filename.endswith(".png"):
        # Read the image
        img_path = os.path.join(PHOTO_FOLDER_PATH, filename)
        img = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)

        # Detect faces in the resized image
        r = detector.detect_faces(img)

        if len(r) > 0:
            # Retrieve the bounding box coordinates
            box = r[0]["box"]

            # Draw the rectangle around the face
            color = (0, 255, 255)  # in BGR
            stroke = 3
            labeled = cv2.rectangle(img, (box[0], box[1]), (box[0] + box[2], box[1] + box[3]), color, stroke)

            # Extract the face region from the original image
            face = labeled[box[1]:box[1] + box[3], box[0]:box[0] + box[2]]

            # Resize the image while maintaining aspect ratio
            max_dim = max(face.shape[:2])
            scale = 640 / max_dim  # Adjust the desired maximum dimension here (currently set to 640)
            new_size = (int(face.shape[1] * scale), int(face.shape[0] * scale))
            face_resized = cv2.resize(face, new_size)
            face_result = cv2.cvtColor(face_resized, cv2.COLOR_RGB2BGR)

            # Save the cut face in the new folder
            cut_face_path = os.path.join(mini_photo_folder, f"{filename[:8]}_face{i}.jpg")
            cv2.imwrite(cut_face_path, face_result)

        else:
            # Add the name of the image to the list if no faces were detected
            no_faces_detected.append(filename)


# Display the images for which no faces were detected
if len(no_faces_detected) > 0:
    print(len(no_faces_detected))
    print("Images with no faces detected:")
    for name in no_faces_detected:
        print(name)
else:
    print("All images have at least one face detected.")



In [ ]:
from pathlib import Path
results_dict = {}

for i, photo_id in enumerate(os.listdir(mini_photo_folder)):
    photo_detections = []
    results = model.predict(source=f"{mini_photo_folder}/{photo_id}", conf = 0.25, save = True)
    for result in results:
        for tensor_id in result:
            bbox, confidence, class_probs = tuple(map(float, tensor_id[:4])), round(float(tensor_id[4]), 5), list(map(int, tensor_id[5:]))
            detection_dict = {
                "bbox": bbox,
                "confidence": confidence,
                "class_probs": class_probs
            }
            
            # Append the result dictionary to the results list
            photo_detections.append(detection_dict)
    results_dict[photo_id] = photo_detections

gitignore_content = """# Ignore all files/directories
*
"""
path_pred = Path("runs")

with open(path_pred / ".gitignore", "w") as gitignore_file:
        gitignore_file.write(gitignore_content)

# Write the results to a JSON file
json_output_path = f"{PROJECT_PATH}/acne_detection_results.json"
with open(json_output_path, "w") as json_file:
    json.dump(results_dict, json_file)